In [37]:
from typing import Any

import numpy as np
import torch
from lightning.pytorch.utilities.types import STEP_OUTPUT

import wandb
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, classification_report
from torch import optim, nn
import torchmetrics
from tqdm import tqdm
import lightning as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger, WandbLogger, CSVLogger

if torch.cuda.is_available():
    print(f'PyTorch version: {torch.__version__}')
    print('*' * 10)
    print(f'_CUDA version: ')
    !nvcc --version
    print('*' * 10)
    print(f'CUDNN version: {torch.backends.cudnn.version()}')
    print(f'Available GPU devices: {torch.cuda.device_count()}')
    print(f'Device Name: {torch.cuda.get_device_name()}')
    device = "cuda"
else:
    device = "cpu"
print(f"Using {device} device")
wandb.login()
logger1 = WandbLogger(project='leaguify', log_model='all')
logger2 = TensorBoardLogger('lightning_logs')
csv_logger = CSVLogger('logs', name='leaguify')

PyTorch version: 2.1.0+cu121
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:09:35_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
**********
CUDNN version: 8801
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 2080
Using cuda device


In [38]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, gru_layers, drop_prob=0.2):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.gru = nn.GRU(input_dim, hidden_dim, gru_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x, h=None):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:, -1]))
        out = nn.Sigmoid()(out)
        return out, h

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(1, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

In [39]:
class LGRU(L.LightningModule):
    def __init__(self, input_dim, hidden_dim, output_dim, gru_layers, drop_prob=0.2):
        super().__init__()
        self.model = GRU(input_dim, hidden_dim, output_dim, gru_layers, drop_prob=drop_prob)
        self.criterion = nn.BCELoss()
        self.save_hyperparameters()
        self.accuracy = torchmetrics.classification.BinaryAccuracy()
        self.f1 = torchmetrics.classification.BinaryF1Score()
        self.confusion_matrix = torchmetrics.classification.BinaryConfusionMatrix()

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.type(torch.float32)
        y = y.type(torch.float32)
        output = self.model(x)[0].squeeze(-1)
        loss = self.criterion(output, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', self.accuracy(output, y), on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = x.type(torch.float32)
        y = y.type(torch.float32)
        output = self.model(x)[0].squeeze(-1)
        loss = self.criterion(output, y)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', self.accuracy(output, y), on_epoch=True, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        x = x.type(torch.float32)
        y = y.type(torch.float32)
        output = self.model(x)[0].squeeze(-1)
        loss = self.criterion(output, y)
        self.log('test_loss', loss, prog_bar=True, logger=True)
        self.log('test_acc', self.accuracy(output, y), prog_bar=True, logger=True)
        self.log('test_f1', self.f1(output, y), prog_bar=True, logger=True)
        #self.log('test_confusion_matrix', self.confusion_matrix(output, y), on_step=True, on_epoch=True, 
        # #prog_bar=True,
        #logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [40]:
class TimelineDataset(Dataset):
    def __init__(self, data_dir, sequence_length, transform=None, target_transform=None):
        self.data = torch.tensor(np.load(data_dir)[:, :-1], dtype=torch.float32, device=device)
        self.labels = torch.tensor(np.load(data_dir)[:, -1], dtype=torch.int64, device=device)
        self.sequence_length = sequence_length
        self.transform = transform
        self.target_transform = target_transform
        self.print_statistics()

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        sample = self.data[idx:idx + self.sequence_length, :]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        if self.target_transform:
            label = self.target_transform(label)
        return sample, label

    def print_statistics(self):
        print(f'Number of samples: {len(self.data)}')
        print(f'Number of features: {len(self.data[0])}')
        print(f'Number of labels: {len(self.labels)}')
        print(f'Number of classes: {len(np.unique(self.labels.cpu().numpy()))}')
        print(f'Number of samples per class: {np.bincount(self.labels.cpu().numpy())}')

In [41]:
def get_train_data(sequence_length=16) -> torch.utils.data.Dataset:
    """
    Get the training data
    :param sequence_length: 
    :return: 
    """
    dataset = TimelineDataset('../data/processed/train_timeline.npy', sequence_length)
    return dataset


def get_val_data(sequence_length=16) -> torch.utils.data.Dataset:
    """
    Get the validation data
    :param sequence_length: 
    :return: 
    """
    dataset = TimelineDataset('../data/processed/val_timeline.npy', sequence_length)
    return dataset


def get_test_data(sequence_length=16) -> torch.utils.data.Dataset:
    """
    Get the test data
    :param sequence_length: 
    :return: 
    """
    dataset = TimelineDataset('../data/processed/test_timeline.npy', sequence_length)
    return dataset

In [42]:
def make_loader(dataset, batch_size=64) -> torch.utils.data.DataLoader:
    """
    Make a data loader
    :param dataset: 
    :param batch_size: 
    :return: 
    """
    return DataLoader(dataset, batch_size=batch_size, num_workers=0, drop_last=False)

In [43]:
print('Training data:')
train_loader = make_loader(get_train_data())
print('Validation data:')
val_loader = make_loader(get_val_data())
print('Test data:')
test_loader = make_loader(get_test_data())

Training data:
Number of samples: 38144
Number of features: 381
Number of labels: 38144
Number of classes: 2
Number of samples per class: [19808 18336]
Validation data:
Number of samples: 12704
Number of features: 381
Number of labels: 12704
Number of classes: 2
Number of samples per class: [6928 5776]
Test data:
Number of samples: 12704
Number of features: 381
Number of labels: 12704
Number of classes: 2
Number of samples per class: [7168 5536]


In [44]:
model = LGRU(input_dim=381, hidden_dim=256, output_dim=1, gru_layers=5, drop_prob=0.2)

In [45]:
trainer = L.Trainer(max_epochs=100, logger=[logger1, logger2, csv_logger], accelerator=device,
                    callbacks=[EarlyStopping(monitor='val_acc', patience=10)])
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)

wandb.finish()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                  | Params
-----------------------------------------------------------
0 | model            | GRU                   | 2.1 M 
1 | criterion        | BCELoss               | 0     
2 | accuracy         | BinaryAccuracy        | 0     
3 | f1               | BinaryF1Score         | 0     
4 | confusion_matrix | BinaryConfusionMatrix | 0     
-----------------------------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.280     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.7310844659805298
         test_f1            0.5848912596702576
        test_loss            0.664667010307312
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇█
test_acc,▁
test_f1,▁
test_loss,▁
train_acc_epoch,▁▂▃▅▅▆▆▇▇██
train_acc_step,▁▆▁▆▃▃▄▆▅▃▆▄█▆▄▃█▁▅█▇▁▆██████▆▆█▇▃█▆▆█▅█
train_loss_epoch,██▇▆▅▅▄▃▃▂▁
train_loss_step,▅▄▄▄▄▄▄▃▄▄▃▄▂▃▄▄▂▇▃▁▂▅▃▂▂▂▂▁▂▄▄▁▂█▂▄▃▂▅▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
val_acc,▁▂▄▇▇▇█▇█▇▇
val_loss,▄▄▃▁▁▂▂▄▃▆█
